# Import data

In [6]:
import pandas as pd

df_covid_AP = pd.read_csv('../lung_pollution/data/covid_pollution_clean.csv')

df_covid_AP.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1'], inplace = True)
df_covid_AP = df_covid_AP.rename(columns={"PM2.5_annualMean": "PM2_5_annualMean", "Fully vaccinated": "Fully_vaccinated", "Population density": "Population_density"})

In [7]:
df_covid_AP.sort_values(['year'], axis=0, ascending=False,inplace=True,ignore_index=True)
df_covid_AP_2019 = df_covid_AP[0:400]

In [8]:
X = df_covid_AP_2019[['NO2_annualMean', 'NO_annualMean', 'O3_annualMean', 'PM10_annualMean','PM2_5_annualMean', 'Fully_vaccinated', 'Population_density']]
y = df_covid_AP_2019[['cases_per_100k']]

# StandardScale

In [26]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()  # Instanciate Robust Scaler

scaler.fit(X)  # Fit scaler to feature
scaled_num = scaler.transform(X)  #Scale
X = pd.DataFrame(scaled_num, columns = X.columns)

# GridSearch

In [35]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

model = RandomForestRegressor()

# Hyperparameter Grid
grid = {'n_estimators': [50, 100, 150, 200], 
       'min_samples_leaf': [1,2,5,10,15],
       'min_samples_split': [1,2,5,10]}

# Instanciate Grid Search
search = GridSearchCV(model, grid, 
                           scoring = 'r2',
                           cv = 5,
                           n_jobs=-1 # paralellize computation
                          ) 
# Fit data to Grid Search
search.fit(X, y)

/Users/dorienroosen/.pyenv/versions/3.8.12/envs/lung_pollution/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
100 fits failed out of a total of 400.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
100 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/dorienroosen/.pyenv/versions/3.8.12/envs/lung_pollution/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/dorienroosen/.pyenv/versions/3.8.12/envs/lung_pollution/lib/python3.8/site-packages/sklearn/ensemble/_forest.py", line 442, in fit
    trees = Parallel(
  File "/Users/dorienroosen/.pyenv/versions

GridSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'min_samples_leaf': [1, 2, 5, 10, 15],
                         'min_samples_split': [1, 2, 5, 10],
                         'n_estimators': [50, 100, 150, 200]},
             scoring='r2')

In [36]:
search.best_score_

0.49886807432626823

In [37]:
search.score

<bound method BaseSearchCV.score of GridSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'min_samples_leaf': [1, 2, 5, 10, 15],
                         'min_samples_split': [1, 2, 5, 10],
                         'n_estimators': [50, 100, 150, 200]},
             scoring='r2')>

In [38]:
search.best_params_

{'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 150}

In [39]:
search.best_estimator_

RandomForestRegressor(min_samples_split=10, n_estimators=150)

In [40]:
forest = RandomForestRegressor(min_samples_leaf= 1, min_samples_split = 2, n_estimators = 150)
forest.fit(X,y)
print('R2 RandomForest: ', forest.score(X,y))


/var/folders/py/ydbm54f573b1ymf9x65hf0mm0000gn/T/ipykernel_81922/510206886.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  forest.fit(X,y)


R2 RandomForest:  0.9354043686514057
